In [1]:
import os
import cv2
import numpy as np

In [3]:
class_names = sorted(os.listdir("train"))

In [4]:
class_names

['AFRICAN LEOPARD',
 'CARACAL',
 'CHEETAH',
 'CLOUDED LEOPARD',
 'JAGUAR',
 'LIONS',
 'OCELOT',
 'PUMA',
 'SNOW LEOPARD',
 'TIGER']

In [5]:
x_train = []
y_train = []

for i, class_name in enumerate(class_names):
    class_dir = os.path.join("train", class_name)
    for filename in os.listdir(class_dir):
        # Load the image and convert it to a numpy array
        img = cv2.imread(os.path.join(class_dir, filename))
        img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
        img_array = np.array(img)

        # Append the numpy array to x_train
        x_train.append(img_array)

        # Append the corresponding label to y_train
        y_train.append(i)

# Convert x_train and y_train to numpy arrays
x_train = np.array(x_train)
y_train = np.array(y_train)

In [6]:
from tensorflow.keras.utils import to_categorical

y_train = to_categorical(y_train, num_classes=10)

In [7]:
class_map = {i: class_names[i] for i in range(len(class_names))}

In [26]:
y_train

array([[1., 0., 0., ..., 0., 0., 0.],
       [1., 0., 0., ..., 0., 0., 0.],
       [1., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 1.],
       [0., 0., 0., ..., 0., 0., 1.],
       [0., 0., 0., ..., 0., 0., 1.]], dtype=float32)

In [8]:
x_val = []
y_val = []

for i, class_name in enumerate(class_names):
    class_dir = os.path.join("valid", class_name)
    for filename in os.listdir(class_dir):
        # Load the image and convert it to a numpy array
        img = cv2.imread(os.path.join(class_dir, filename))
        img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
        img_array = np.array(img)

        # Append the numpy array to x_train
        x_val.append(img_array)

        # Append the corresponding label to y_train
        y_val.append(i)

# Convert x_train and y_train to numpy arrays
x_val = np.array(x_val)
y_val = np.array(y_val)

array([[[[0.87843137, 0.51764706, 0.23137255],
         [0.84313725, 0.48235294, 0.19607843],
         [0.8745098 , 0.50980392, 0.23529412],
         ...,
         [0.85490196, 0.47058824, 0.21568627],
         [0.89019608, 0.49019608, 0.23137255],
         [0.85882353, 0.45882353, 0.2       ]],

        [[0.84313725, 0.48235294, 0.19607843],
         [0.83529412, 0.4745098 , 0.18823529],
         [0.85490196, 0.49019608, 0.21568627],
         ...,
         [0.87843137, 0.49411765, 0.23921569],
         [0.85098039, 0.4627451 , 0.2       ],
         [0.84705882, 0.44705882, 0.18823529]],

        [[0.92156863, 0.56078431, 0.2745098 ],
         [0.87843137, 0.51764706, 0.23137255],
         [0.84313725, 0.47843137, 0.20392157],
         ...,
         [0.82745098, 0.44313725, 0.18823529],
         [0.84705882, 0.45882353, 0.19607843],
         [0.84313725, 0.45490196, 0.19215686]],

        ...,

        [[0.73333333, 0.37647059, 0.16078431],
         [0.8627451 , 0.5372549 , 0.34117647]

In [9]:
y_val = to_categorical(y_val, num_classes=10)


In [16]:
import numpy as np
from sklearn.model_selection import StratifiedShuffleSplit

# Concatenate the x and y arrays
x_all = np.concatenate([x_train, x_val], axis=0)
y_all = np.concatenate([y_train, y_val], axis=0)

# Create a StratifiedShuffleSplit object
sss = StratifiedShuffleSplit(n_splits=1, test_size=0.2, random_state=42)

# Use the StratifiedShuffleSplit object to split the data
train_indices, test_indices = next(sss.split(x_all, y_all))

# Split the data into training and testing sets
x_train_new, x_test_new = x_all[train_indices], x_all[test_indices]
y_train_new, y_test_new = y_all[train_indices], y_all[test_indices]

# Verify the shapes of the new arrays
print(f'Shape of x_train_new: {x_train_new.shape}')
print(f'Shape of y_train_new: {y_train_new.shape}')
print(f'Shape of x_test_new: {x_test_new.shape}')
print(f'Shape of y_test_new: {y_test_new.shape}')


Shape of x_train_new: (1911, 224, 224, 3)
Shape of y_train_new: (1911, 10)
Shape of x_test_new: (478, 224, 224, 3)
Shape of y_test_new: (478, 10)


In [38]:
from tensorflow.keras.applications import VGG16
from tensorflow.keras.layers import Input, Flatten, Dense
from tensorflow.keras.models import Model

# Load the pre-trained VGG16 model
vgg_model = VGG16(weights='imagenet', include_top=False, input_tensor=Input(shape=(224, 224, 3)))

# Freeze the layers in the pre-trained model
for layer in vgg_model.layers:
    layer.trainable = False

# Add your own classification layers
x = Flatten()(vgg_model.output)
x = Dense(512, activation='relu')(x)
x = Dense(256, activation='relu')(x)
predictions = Dense(10, activation='softmax')(x)

# Create the new model
model = Model(inputs=vgg_model.input, outputs=predictions)

# Compile the model
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
history = model.fit(x_train_new, y_train_new, epochs=10, batch_size=32, validation_data=(x_test_new, y_test_new))
model.save("Models/base_vgg16.h5")

Epoch 1/10
60/60 [==============================] - 365s 6s/step - loss: 14.5336 - accuracy: 0.6033 - val_loss: 4.1302 - val_accuracy: 0.7448
Epoch 2/10
60/60 [==============================] - 395s 7s/step - loss: 1.2534 - accuracy: 0.8943 - val_loss: 3.1559 - val_accuracy: 0.7887
Epoch 3/10
60/60 [==============================] - 397s 7s/step - loss: 0.4695 - accuracy: 0.9560 - val_loss: 3.6325 - val_accuracy: 0.7720
Epoch 4/10
60/60 [==============================] - 404s 7s/step - loss: 0.2158 - accuracy: 0.9744 - val_loss: 3.3122 - val_accuracy: 0.7887
Epoch 5/10
60/60 [==============================] - 392s 7s/step - loss: 0.2639 - accuracy: 0.9707 - val_loss: 3.2307 - val_accuracy: 0.8285
Epoch 6/10
60/60 [==============================] - 392s 7s/step - loss: 0.1397 - accuracy: 0.9822 - val_loss: 2.2215 - val_accuracy: 0.8640
Epoch 7/10
60/60 [==============================] - 392s 7s/step - loss: 0.1939 - accuracy: 0.9765 - val_loss: 3.1981 - val_accuracy: 0.8180
Epoch 8/10
6

In [39]:
from tensorflow.keras.applications.vgg16 import VGG16
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Flatten
from tensorflow.keras.utils import to_categorical
from sklearn.model_selection import train_test_split
import numpy as np
y_train = np.asarray(y_train)
y_val = np.asarray(y_val)
# Load the pre-trained VGG16 model without the top layer
vgg_model = VGG16(weights='imagenet', include_top=False, input_shape=(224, 224, 3))

# Freeze the pre-trained layers so they are not updated during training
for layer in vgg_model.layers:
    layer.trainable = False

# Define a new model that includes the VGG16 model followed by a few additional layers
model = Sequential()
model.add(vgg_model)
model.add(Flatten())
model.add(Dense(256, activation='relu'))
model.add(Dense(10, activation='softmax'))

# Compile the model
model.compile(optimizer='adam',
              loss='categorical_crossentropy',
              metrics=['accuracy'])


# Fit the model with validation data
history = model.fit(x_train_new, y_train_new, epochs=10, batch_size=32, validation_data=(x_test_new, y_test_new))
model.save("Models/vgg16_tl.h5")


Epoch 1/10
60/60 [==============================] - 396s 7s/step - loss: 17.5772 - accuracy: 0.6337 - val_loss: 3.4935 - val_accuracy: 0.7594
Epoch 2/10
60/60 [==============================] - 394s 7s/step - loss: 0.6574 - accuracy: 0.9246 - val_loss: 2.2084 - val_accuracy: 0.8368
Epoch 3/10
60/60 [==============================] - 395s 7s/step - loss: 0.2767 - accuracy: 0.9665 - val_loss: 2.4765 - val_accuracy: 0.8347
Epoch 4/10
60/60 [==============================] - 395s 7s/step - loss: 0.2296 - accuracy: 0.9723 - val_loss: 3.1954 - val_accuracy: 0.8201
Epoch 5/10
60/60 [==============================] - 396s 7s/step - loss: 0.2284 - accuracy: 0.9754 - val_loss: 2.8430 - val_accuracy: 0.8305
Epoch 6/10
60/60 [==============================] - 398s 7s/step - loss: 0.1156 - accuracy: 0.9838 - val_loss: 2.6585 - val_accuracy: 0.8452
Epoch 7/10
60/60 [==============================] - 398s 7s/step - loss: 0.0590 - accuracy: 0.9906 - val_loss: 2.8106 - val_accuracy: 0.8368
Epoch 8/10
6

In [ ]:
# import cv2
# import numpy as np

# # Apply image sharpening
# kernel = np.array([[-1,-1,-1], [-1,9,-1], [-1,-1,-1]]) # define a sharpening kernel
# sharpened_train = np.zeros_like(x_train) # create an empty array to store the sharpened images
# for i in range(x_train.shape[0]):
#     img = x_train[i]
#     sharpened = cv2.filter2D(img, -1, kernel)
#     sharpened_train[i] = sharpened

# # Apply histogram equalization
# equalized_train = np.zeros_like(sharpened_train) # create an empty array to store the equalized images
# for i in range(sharpened_train.shape[0]):
#     img = sharpened_train[i]
#     img_yuv = cv2.cvtColor(img, cv2.COLOR_BGR2YUV)
#     img_yuv[:,:,0] = cv2.equalizeHist(img_yuv[:,:,0])
#     equalized = cv2.cvtColor(img_yuv, cv2.COLOR_YUV2BGR)
#     equalized_train[i] = equalized


In [17]:
import cv2
import numpy as np

# Define the sharpening kernel
kernel = np.array([[-1,-1,-1],[-1,9,-1],[-1,-1,-1]])

# Apply histogram equalization and sharpening to x_train_new
for i in range(len(x_train_new)):
    # Convert image to grayscale
    gray = cv2.cvtColor(x_train_new[i], cv2.COLOR_BGR2GRAY)
    
    # Apply histogram equalization
    equalized = cv2.equalizeHist(gray)
    
    # Apply sharpening
    sharpened = cv2.filter2D(equalized, -1, kernel)
    
    # Convert back to BGR color space
    x_train_new[i] = cv2.cvtColor(sharpened, cv2.COLOR_GRAY2BGR)

# Apply histogram equalization and sharpening to x_test_new
for i in range(len(x_test_new)):
    # Convert image to grayscale
    gray = cv2.cvtColor(x_test_new[i], cv2.COLOR_BGR2GRAY)
    
    # Apply histogram equalization
    equalized = cv2.equalizeHist(gray)
    
    # Apply sharpening
    sharpened = cv2.filter2D(equalized, -1, kernel)
    
    # Convert back to BGR color space
    x_test_new[i] = cv2.cvtColor(sharpened, cv2.COLOR_GRAY2BGR)


In [41]:
from tensorflow.keras.applications.vgg16 import VGG16
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Flatten
from tensorflow.keras.utils import to_categorical
from sklearn.model_selection import train_test_split
import numpy as np
y_train = np.asarray(y_train)
y_val = np.asarray(y_val)
# Load the pre-trained VGG16 model without the top layer
vgg_model = VGG16(weights='imagenet', include_top=False, input_shape=(224, 224, 3))

# Freeze the pre-trained layers so they are not updated during training
for layer in vgg_model.layers:
    layer.trainable = False

# Define a new model that includes the VGG16 model followed by a few additional layers
model = Sequential()
model.add(vgg_model)
model.add(Flatten())
model.add(Dense(256, activation='relu'))
model.add(Dense(10, activation='softmax'))

# Compile the model
model.compile(optimizer='adam',
              loss='categorical_crossentropy',
              metrics=['accuracy'])


# Fit the model with validation data
history = model.fit(x_train_new, y_train_new, epochs=10, batch_size=32, validation_data=(x_test_new, y_test_new))
model.save("Models/enhanced_vgg16_tl.h5")

Epoch 1/10
60/60 [==============================] - 405s 7s/step - loss: 22.1060 - accuracy: 0.3339 - val_loss: 1.9463 - val_accuracy: 0.2908
Epoch 2/10
60/60 [==============================] - 404s 7s/step - loss: 1.2534 - accuracy: 0.5824 - val_loss: 1.6595 - val_accuracy: 0.4812
Epoch 3/10
60/60 [==============================] - 401s 7s/step - loss: 0.7053 - accuracy: 0.7666 - val_loss: 1.7185 - val_accuracy: 0.5481
Epoch 4/10
60/60 [==============================] - 395s 7s/step - loss: 0.4485 - accuracy: 0.8482 - val_loss: 1.5503 - val_accuracy: 0.6151
Epoch 5/10
60/60 [==============================] - 395s 7s/step - loss: 0.2783 - accuracy: 0.9131 - val_loss: 1.6622 - val_accuracy: 0.6109
Epoch 6/10
60/60 [==============================] - 396s 7s/step - loss: 0.1476 - accuracy: 0.9597 - val_loss: 1.7181 - val_accuracy: 0.6339
Epoch 7/10
60/60 [==============================] - 396s 7s/step - loss: 0.0906 - accuracy: 0.9712 - val_loss: 1.8960 - val_accuracy: 0.6172
Epoch 8/10
6

In [21]:
x_train[4]

array([[[172, 152,  81],
        [170, 154,  79],
        [168, 156,  80],
        ...,
        [133,  95,  46],
        [135,  94,  38],
        [165, 123,  63]],

       [[155, 140,  71],
        [171, 159,  87],
        [176, 170,  94],
        ...,
        [130,  92,  43],
        [142, 103,  44],
        [168, 128,  66]],

       [[148, 143,  77],
        [137, 134,  65],
        [134, 137,  66],
        ...,
        [131,  91,  40],
        [142, 106,  44],
        [160, 125,  59]],

       ...,

       [[173, 152,  97],
        [217, 198, 155],
        [156, 146, 121],
        ...,
        [ 70,  74,  57],
        [ 84,  88,  71],
        [ 67,  74,  56]],

       [[193, 165, 115],
        [221, 196, 155],
        [232, 211, 190],
        ...,
        [ 55,  58,  39],
        [ 67,  69,  48],
        [ 58,  60,  38]],

       [[207, 177, 127],
        [228, 200, 160],
        [243, 218, 198],
        ...,
        [ 62,  65,  46],
        [ 70,  69,  48],
        [ 68,  68,  44]]